# KEC Educational Content Analysis with Machine Learning
## Web Scraping and Classification System for Form 1-4 Content

This notebook scrapes educational content from https://kec.ac.ke/ and builds ML models for content classification and question answering.

In [1]:
# Install required packages
!pip install requests beautifulsoup4 selenium pandas numpy scikit-learn nltk transformers torch
!pip install webdriver-manager lxml matplotlib seaborn wordcloud

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 929.6 kB/s eta 0:00:10
   --- ------------------------------------ 0.8/9.6 MB 958.5 kB/s eta 0:00:10
   ---- ----------------------------------- 1.0/9.6 MB 986.7 kB/s eta 0:00:09
   ----- ---------------------------------- 1.3/9.6 MB 1.0 MB/s eta 0:00:09
   ------ --------------------------------- 1.6/9.6 MB 1.0 MB/s eta 0:00:08
   ------ --------------------------------- 1.6/9.6 MB 1.0 MB/s eta 0:00:08
   ------- -------------------------------- 1.8/9.6 MB 1.0 MB/s eta 0:00:08
   -------- ------------------------------- 2.1/9.6 MB 1.1 MB/s eta 0:00:08
   ---------- ------------------


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.0 MB 1.8 MB/s eta 0:00:02
   ------------- -------------------------- 1.3/4.0 MB 1.9 MB/s eta 0:00:02
   -------------------- ------------------- 2.1/4.0 MB 2.3 MB/s eta 0:00:01
   ---------------------------- ----------- 2.9/4.0 MB 2.6 MB/s eta 0:00:01
   ------------------------------------ --- 3.7/4.0 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 5.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.3/8.1 MB 3.7 MB/s eta 0:00:02
   ---------- ------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
from urllib.parse import urljoin, urlparse
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import warnings
warnings.filterwarnings('ignore')

In [3]:
# ML and NLP imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\Denno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Denno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Denno/nltk_data...
[nltk_data] Downloading package omw-1.4 to C:\Users\Denno/nltk_data...


True

## 1. Web Scraping KEC Website

In [5]:
class KECWebScraper:
    def __init__(self):
        self.base_url = "https://kec.ac.ke/"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        self.scraped_data = []
        
    def setup_selenium_driver(self):
        """Setup Selenium WebDriver for dynamic content"""
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        service = Service(ChromeDriverManager().install())
        return webdriver.Chrome(service=service, options=options)
    
    def scrape_main_page(self):
        """Scrape the main KEC page to find educational content links"""
        try:
            response = self.session.get(self.base_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find education level sections
            education_sections = {
                'pre_primary': [],
                'primary': [],
                'junior_secondary': [],
                'secondary': []
            }
            
            # Look for grade/form links
            links = soup.find_all('a', href=True)
            for link in links:
                href = link.get('href')
                text = link.get_text().lower().strip()
                
                if any(grade in text for grade in ['form 1', 'form 2', 'form 3', 'form 4']):
                    education_sections['secondary'].append({
                        'url': urljoin(self.base_url, href),
                        'title': link.get_text().strip(),
                        'level': 'secondary'
                    })
            
            return education_sections
        except Exception as e:
            print(f"Error scraping main page: {e}")
            return {}
    
    def scrape_content_page(self, url, title, level):
        """Scrape individual content pages"""
        try:
            response = self.session.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract text content
            content_text = ""
            
            # Remove script and style elements
            for script in soup(["script", "style"]):
                script.decompose()
            
            # Get text content
            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            content_text = ' '.join(chunk for chunk in chunks if chunk)
            
            return {
                'url': url,
                'title': title,
                'level': level,
                'content': content_text,
                'word_count': len(content_text.split()),
                'scraped_at': time.strftime('%Y-%m-%d %H:%M:%S')
            }
        except Exception as e:
            print(f"Error scraping {url}: {e}")
            return None
    
    def scrape_all_content(self):
        """Main method to scrape all educational content"""
        print("Starting KEC website scraping...")
        
        # Get main page structure
        sections = self.scrape_main_page()
        
        # Scrape each section
        for section_name, links in sections.items():
            print(f"Scraping {section_name} section...")
            for link_info in links:
                content = self.scrape_content_page(
                    link_info['url'], 
                    link_info['title'], 
                    link_info['level']
                )
                if content:
                    self.scraped_data.append(content)
                time.sleep(1)  # Be respectful to the server
        
        return self.scraped_data

In [6]:
# Initialize and run scraper
scraper = KECWebScraper()
scraped_content = scraper.scrape_all_content()

print(f"Scraped {len(scraped_content)} pages of content")
if scraped_content:
    df_content = pd.DataFrame(scraped_content)
    print(df_content.head())

Starting KEC website scraping...
Scraping pre_primary section...
Scraping primary section...
Scraping junior_secondary section...
Scraping secondary section...
Scraped 4 pages of content
                                                 url   title      level  \
0  https://lms.kec.ac.ke/course/index.php?categor...  Form 1  secondary   
1  https://lms.kec.ac.ke/course/index.php?categor...  Form 2  secondary   
2  https://lms.kec.ac.ke/course/index.php?categor...  Form 3  secondary   
3  https://lms.kec.ac.ke/course/index.php?categor...  Form 4  secondary   

                                             content  word_count  \
0  Digital Content | Kenya Education Cloud | Unli...        5375   
1  Digital Content | Kenya Education Cloud | Unli...        5375   
2  Digital Content | Kenya Education Cloud | Unli...        5375   
3  Digital Content | Kenya Education Cloud | Unli...        5375   

            scraped_at  
0  2025-09-07 23:50:27  
1  2025-09-07 23:50:28  
2  2025-09-07 23:50:3